<a href="https://colab.research.google.com/github/SamuelGeron/gcpDW/blob/main/Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Payments Data

## Import and Install Google Drive Libraries

In [ ]:
#Imports and install
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Authenticates

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Get Data File

In [6]:
file_link = 'https://drive.google.com/open?id=1GlYrv7ex0ClxQwQ0NvJ4GTUGre7s8vtw'
fluff, id = file_link.split('=')
pagamentos_file = drive.CreateFile({'id':id}) 
pagamentos_file.GetContentFile('pagamentos.csv')  

## Store Data into Big Query

In [14]:
import pandas as pd
from pandas.io import gbq
pagamentos = pd.read_csv('pagamentos.csv',names=["ClienteID", "DataDoPagamento", "Plano", "Valor"])

In [16]:
#Connect and ingest data into extract dataset
pagamentos.to_gbq(destination_table='extract.pagamentos',
                  project_id='animated-falcon-292800',
                  if_exists='replace')

1it [00:05,  5.04s/it]


# Client Data

## Import Libraries

In [19]:
import urllib3
from urllib3 import request
import certifi
import json

## Certicate Handling

In [22]:
# handle certificate verification and SSL warnings
# https://urllib3.readthedocs.io/en/latest/user-guide.html#ssl
http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

## Retrieve Data

In [ ]:
api_endpoint = 'https://demo0192734.mockable.io/clientes/'
rest_output = http.request('GET', api_endpoint)
clientes = json.loads(rest_output.data.decode('utf-8'))
clientes_df = pd.json_normalize(clientes)
clientes_df

## Store Data into BigQuery

In [26]:
clientes_df.to_gbq(destination_table='extract.clientes',
                  project_id='animated-falcon-292800',
                  if_exists='replace')

1it [00:02,  2.57s/it]
